In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import re
import string
import json
import keras
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

In [ ]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
X=X['title']+' '+X['text']

X = X.apply(clean_text)

In [ ]:
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

In [ ]:
clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

In [ ]:
fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

In [ ]:
fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

In [ ]:
real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)

In [ ]:
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
to_txt=x_train+x_test

In [ ]:
y=y_train+y_test

In [ ]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [ ]:
#수동으로 cross validation을 하기위한 코드
cnt=2  #cnt가 2일때 두번째 시행
cnt=cnt-2

def exclude_list(input_list, cnt):
    return input_list[:cnt*4000]+input_list[cnt*4000+4000:16000]

if cnt>-1:
    x_train_cnt=exclude_list(x_train,cnt)+x_test
    y_train_cnt=exclude_list(y_train,cnt)+y_test
    x_test_cnt=x_train[cnt*4000:cnt*4000+4000]
    y_test_cnt=y_train[cnt*4000:cnt*4000+4000]

In [ ]:
if cnt>-1:
    x_train=x_train_cnt
    y_train=y_train_cnt
    x_test=x_test_cnt
    y_test=y_test_cnt

In [ ]:
lst=[]
for i in range(16000):
    
    lst.append(len(x_train[i].split()))

In [ ]:
max(lst)

In [ ]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 1000

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    
        
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm
    
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
    
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            'max_len': self.max_len,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config


embedding_dim = 128  # 각 단어의 임베딩 벡터의 차원 
num_heads = 1  # 어텐션 헤드의 수
dff = 128 # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기 
num_transformer_blocks = 1

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    x = transformer_block(x)

# transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
# x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.summary()

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
tmp = [[x,y] for x, y in zip(x_train, y_train)]
import random
random.shuffle(tmp)
x_train = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
x_val=x_train[:1000]
x_train=x_train[1000:]
y_val=y_train[:1000]
y_train=y_train[1000:]

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=128,callbacks=callbacks, epochs=10, validation_data=(x_val, y_val))

print("테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [ ]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_5.h5', custom_objects=custom_objects)
model.evaluate(x_test, y_test)